# Scrape Topic Details
[Funding & Tenders Portal Application Programming Interfaces (APIs)](https://ec.europa.eu/info/funding-tenders/opportunities/portal/screen/support/apis)

In [1]:
import json
import requests
import pandas as pd

# name = 'horizon-cl4-2022-data-01-04'
# name = 'horizon-cl5-2022-d6-02-07'
# name = 'horizon-jti-cleanh2-2022-01-03'
# name = 'horizon-eie-2022-scaleup-02-01'
name = 'erasmus-edu-2022-pi-all-inno-edu-enterp'

topicDetailsURL = 'https://ec.europa.eu/info/funding-tenders/opportunities/data/topicDetails/'+name+'.json'
topic = json.loads(requests.get(topicDetailsURL).text)['TopicDetails']
topic.keys()

dict_keys(['type', 'ccm2Id', 'cftId', 'identifier', 'title', 'publicationDateLong', 'callIdentifier', 'callTitle', 'callccm2Id', 'allowPartnerSearch', 'workProgrammepart', 'frameworkProgramme', 'programmeDivision', 'topicMGAs', 'keywords', 'sme', 'actions', 'latestInfos', 'budgetOverviewJSONItem', 'description', 'conditions', 'supportInfo', 'sepTemplate', 'links', 'additionalDossiers', 'infoPackDossiers', 'callDetailsJSONItem'])

In [2]:
basicInfoMapping = {
    "Submission deadline": topic["actions"][0]['deadlineDates'][0],
    "Topic name" : topic["title"],
    "Topic ID" : topic["identifier"]
    }
for k,v in basicInfoMapping.items():
    print(k + ': ' + v)

Submission deadline: 15 September 2022
Topic name: Alliances for Education and Enterprises
Topic ID: ERASMUS-EDU-2022-PI-ALL-INNO-EDU-ENTERP


In [3]:
'HORIZON-EIE-2022-SCALEUP-02-01'.lower()

'horizon-eie-2022-scaleup-02-01'

In [4]:
from bs4 import BeautifulSoup
template = """<!DOCTYPE html><html><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- <link href="styles.css" rel="stylesheet"/> -->
<style>
body { 
    background-color: #ffffff; 
    color: #000000; 
    margin:25px 25px 25px 25px; 
    font-family: Arial, Helvetica, sans-serif; 
}
.frame {
    background-color: #cceeff;
    border-style: solid; 
    border-color: #003199; 
    padding: 10px
}
.red { background-color: #ffcccc;}
.peach { background-color: #ffeecc;}
.cream, .highlight { background-color: #ffffcc;}
.green { background-color: #ccffcc;}
.onahau { background-color: #ccffff;}
.blue { background-color: #cceeff;}
.periwinkle { background-color: #ccccff;}
table, th, td {
    background-color: #cceeff;
    border: 1px solid #003199;
    border-collapse: collapse;
}
th, td {
    padding: 5px;
}
p.MsoNormal, li.MsoNormal, div.MsoNormal {
    margin-top:0cm;
    margin-right:0cm;
    margin-bottom:8.0pt;
    margin-left:0cm;
    line-height:107%;
    font-size:11.0pt;
    font-family:"Calibri",sans-serif;
}
h1 {
    mso-style-link:"Nadpis 1 Char";
    color:#2E74B5;
}
h2 {
    mso-style-link:"Nadpis 2 Char";
    color:#2E74B5;
}
h3 {
    mso-style-link:"Nadpis 3 Char";
    color:#2E74B5;
}
p.MsoNoSpacing, li.MsoNoSpacing, div.MsoNoSpacing {
    margin:0cm;
    margin-bottom:.0001pt;
    font-size:11.0pt;
    font-family:"Calibri",sans-serif;
}
</style>
<title></title></head>
<body></body></html>
"""

# Initialize the template
soup = BeautifulSoup(template, 'html.parser')
# Add title and ID
h = BeautifulSoup('<h1></h1>', 'html.parser').h1
h.append(BeautifulSoup(topic["title"], 'html.parser'))
soup.body.append(h)

# Add General information
links = topic['links']
if len(links) > 0:
    links = topic['links'][0]
    table = BeautifulSoup('<table></table>', 'html.parser').table

    tr = BeautifulSoup('<tr></tr>', 'html.parser').tr
    td = BeautifulSoup('<td style="width:10%">Topic ID</td>', 'html.parser').td
    tr.append(td)
    td = BeautifulSoup('<td></td>', 'html.parser').td
    a = BeautifulSoup('<a></a>', 'html.parser').a
    a['href'] = links['url']
    a.append(topic["identifier"])
    td.append(a)
    tr.append(td)
    table.append(tr)

    tr = BeautifulSoup('<tr></tr>', 'html.parser').tr
    td = BeautifulSoup('<td>Opening date</td>', 'html.parser').td
    tr.append(td)
    td = BeautifulSoup('<td></td>', 'html.parser').td
    td.append(links['startDate'])
    tr.append(td)
    table.append(tr)

    tr = BeautifulSoup('<tr></tr>', 'html.parser').tr
    td = BeautifulSoup('<td>Deadline date</td>', 'html.parser').td
    tr.append(td)
    td = BeautifulSoup('<td></td>', 'html.parser').td
    b = BeautifulSoup('<b></b>', 'html.parser').b
    b.append(links['endDate'])
    td.append(b)
    tr.append(td)
    table.append(tr)

    note = '<p>Please submit your proposal at the latest <b>48 hours before the deadline</b> (to avoid system congestion or system incompatibility issues). Late proposals will not be accepted. Don’t wait until the end because you are worried about confidentiality – we will not open the proposals before the deadline. You can submit the proposal as many times as you wish up until the deadline. Each new submitted version will replace the previous one.</p>'
    tr = BeautifulSoup('<tr></tr>', 'html.parser').tr
    td = BeautifulSoup('<td>Note</td>', 'html.parser').td
    tr.append(td)
    td = BeautifulSoup('<td></td>', 'html.parser').td
    td.append(BeautifulSoup(note, 'html.parser').p)
    tr.append(td)
    table.append(tr)

    tr = BeautifulSoup('<tr></tr>', 'html.parser').tr
    td = BeautifulSoup('<td>Type of MGA</td>', 'html.parser').td
    tr.append(td)
    td = BeautifulSoup('<td></td>', 'html.parser').td
    td.append(links['mgaDescription']+' ['+links['mgaCode']+']')
    tr.append(td)
    table.append(tr)

    soup.body.append(table)

# Add table of contents
p = BeautifulSoup('<p><b>Table of Contents: </b></p>', 'html.parser').p
p.append('[')
a = BeautifulSoup('<a>Topic Description</a>', 'html.parser').a
a['href'] = '#description'
p.append(a)
p.append('] [')
a = BeautifulSoup('<a>Destination</a>', 'html.parser').a
a['href'] = '#destination'
p.append(a)
p.append('] [')
a = BeautifulSoup('<a>Topic Conditions and Documents</a>', 'html.parser').a
a['href'] = '#conditions'
p.append(a)
p.append(']')
soup.body.append(p)
# Add topic description
h = BeautifulSoup('<h2>Topic Description</h2>', 'html.parser').h2
h['id'] = 'description'
soup.body.append(h)
item = BeautifulSoup(topic['description'], 'html.parser')
soup.body.append(item)

# Add destination details
try:
    h = BeautifulSoup('<h2>Destination: </h2>', 'html.parser').h2
    h['id'] = 'destination'
    h.append(BeautifulSoup(topic["destinationDescription"], 'html.parser'))
    soup.body.append(h)

    item = BeautifulSoup(topic['destinationDetails'], 'html.parser')
    soup.body.append(item)
except:
    pass

# Add Conditions
h = BeautifulSoup('<h2>Topic Conditions and Documents</h2>', 'html.parser').h2
h['id'] = 'conditions'
soup.body.append(h)

item = BeautifulSoup(topic['conditions'], 'html.parser')
soup.body.append(item)

# Add last change date
latestInfos = topic['callDetailsJSONItem']['latestInfos']
if len(latestInfos) > 0:
    p = BeautifulSoup('<p></p>', 'html.parser').p
    p['class'] = 'frame'
    span = BeautifulSoup('<span>Last change date: <b></b></span>', 'html.parser').span
    span.b.append(BeautifulSoup(topic['callDetailsJSONItem']['latestInfos'][0]['lastChangeDate'], 'html.parser'))
    p.append(span)
    soup.body.append(p)

# Save the output
with open('./html-out/'+name+'_.html', 'wb') as file:
    file.write(soup.encode())

In [5]:
# topic["callTitle"]
# topic["links"]
# topic['callDetailsJSONItem'] #['latestInfos'] #[0]['lastChangeDate']
topic["destinationDescription"]

KeyError: 'destinationDescription'